# Exercise 2: Swendsen-Wang algorithm for the 2D Ising model

The goal of this exercise is to implement the Swendsen-Wang algorithm for the 2D Ising
model. Recall that the update of the Swendsen-Wang algorithm works as follows:

- Assign to each bond $b$ a variable $w_b \in \{ 0, 1 \}$, where 1 means “connected” and 0
means “disconnected”. If the spins connected by the bond $b = (n, m)$ between index
$n$ and $m$ are anti-parallel, always choose $w_b = 0$, If the spins are parallel, choose
$w_b = 0$ with probability $e^{−2\Beta J} , where $\Beta = k_bT$. In terms of conditional probabilities:

\begin{align}
p(w_b = 0 | \sigma_n \neq \sigma_m) &= 1 \\
p(w_b = 0 | \sigma_n = \sigma_m) &= e^{−2\Beta J} \\
p(w_b = 1 | \sigma_n \neq \sigma_m) &= 0 \\
p(w_b = 1 | \sigma_n = \sigma_m) &= 1 − e^{−2\Beta J} \\
\end{align}

- Interpreting the connected bonds as edges of a graph (where 0 means no edge) and
the spins as node, find “clusters” of spins, i.e., connected components of the graph.
- Flip each cluster (= connected component) with probability $p = 0.5$. Notice that a
single spin is also a “cluster”.
After the update, do a measurement (if the system is already thermalized) and continue
with the next update

In [ ]:
%matplotlib widget
import numpy as np
from matplotlib import pyplot as plt
import numba
from numba import prange, float32, int32, int8
from scipy import sparse as sp
plt.rcParams["animation.html"] = "jshtml"

In [ ]:
spec = {
    "J": float32,
    "Lx": int32,
    "Ly": int32,
    "bond_indices": int32[:, :],
    "spins": int8[:, :],
    "bonds": int8[:],
}

# @numba.experimental.jitclass(spec)
class System:

    def __init__(self, J: float, Lx: int, Ly: int, initial_state: np.ndarray = None):
        self.J = J
        self.Lx = Lx
        self.Ly = Ly

        # Initialize bond indices
        self.bond_indices = np.empty((2*self.N, 2), dtype=np.int32)
        for n in range(self.N):
            x, y = self.idx_2_xy(n)
            # left-right
            self.bond_indices[2*n, 0] = n
            self.bond_indices[2*n, 1] = self.xy_2_idx(x, (y + 1) % self.Ly)
            # up-down
            self.bond_indices[2*n + 1, 0] = n
            self.bond_indices[2*n + 1, 1] = self.xy_2_idx((x + 1) % self.Lx, y)

        # Set random initial state if not given.
        # Numba does not support many array operations,
        # so this algorithm is dumbed down a lot.
        self.spins = np.ones(self.shape, dtype=np.int8)
        if initial_state is None:
            for n in prange(self.N):
                if np.random.random() < 0.5:
                    y, x = self.idx_2_xy(n)
                    self.spins[y, x] = -1
        else:
            self.spins = initial_state

        self.bonds = np.zeros(2*self.N, dtype=np.int8)
            
    @property
    def N(self) -> int:
        return self.Lx * self.Ly
    
    @property
    def shape(self) -> tuple[int, int]:
        return self.Ly, self.Lx
    
    def xy_2_idx(self, x: int, y: int) -> int:
        return self.Ly * x + y
    
    def idx_2_xy(self, idx: int) -> tuple[int, int]:
        return divmod(idx, self.Ly)
    
    def get_spin(self, idx: int) -> int:
        return self.spins[self.idx_2_xy(idx)]
    
    def update_bonds(self, kbT: float):
        for i, (spin_1, spin_2) in enumerate(self.bond_indices):
            # equal spins: no bond
            if self.get_spin(spin_1) != self.get_spin(spin_2):
                self.bonds[i] = 0
            # Different spins: 1 with probability e^-beta*J
            else:
                self.bonds[i] = np.random.random() > np.exp(-self.J / kbT)
    
    def flip_clusters(self, n_clusters: int, clusters: np.ndarray):
        # pre-determine which clusters to flip
        clusters_to_flip = np.random.random(n_clusters) < 0.5
        # loop through all the spins
        for idx in prange(self.N):
            # check if it should be flipped
            if clusters_to_flip[clusters[idx]]:
                # flip
                self.spins[self.idx_2_xy(idx)] = -self.spins[self.idx_2_xy(idx)]
    
    def show_clusters(self, clusters: np.ndarray) -> np.ndarray:
        out = np.empty(self.shape)
        for idx in prange(self.N):
            out[self.idx_2_xy(idx)] = clusters[idx]
        return out
    
    @property
    def E(self) -> float:
        out = 0
        for spin_1, spin_2 in self.bond_indices:
            if self.get_spin(spin_1) != self.get_spin(spin_2):
                out += self.J
            else:
                out -= self.J
        return out
    
    @property
    def M(self) -> float:
        return np.mean(self.spins)
            

# scipy is not supported in jit classes
# I can't be bothered to rewrite it now
def find_connected_components(s: System):
    graph = sp.csr_matrix(
        (s.bonds, (s.bond_indices[:, 0], s.bond_indices[:, 1])),
        dtype=np.int8,
        shape=(s.N, s.N)
        )
    graph.eliminate_zeros()
    graph += graph.T
    connections = sp.csgraph.connected_components(graph)
    return connections

def iterate_system(s: System, kbT: float):
    s.update_bonds(kbT)
    s.flip_clusters(*find_connected_components(s))

In [ ]:
# Animation to verify
from matplotlib.animation import FuncAnimation
from matplotlib.widgets import Slider
s = System(1.0, 30, 30)
T = 1.5
frames = 100

clusters = lambda : s.show_clusters(find_connected_components(s)[1])


fig, (ax1, ax2) = plt.subplots(2,1)
ax1.set_xticks([])
ax1.set_yticks([])
im1 = ax1.matshow(s.spins)
# ax2.spines["top"].set_visible(True)
# ax2.spines["right"].set_visible(True)
T_slider = Slider(ax2, 'Temperature ', valmin=0, valmax=5, 
             valinit=1, valfmt='%i K', facecolor='#cc7000')
fig.tight_layout()

def animation(frame):
    iterate_system(s, T)
    im1.set_data(s.spins)
    return im1,

def change_T(_):
    global T
    T = T_slider.val
T_slider.on_changed(change_T)

FuncAnimation(fig, animation, frames=frames, blit=False, interval=50)


In [ ]:
s = System(1.0, 10, 10)
T_range = np.arange(1.5, 3.1, 0.1)

N_sweeps = 10     # Number of steps for the measurements
N_eq = 100         # Number of equilibration steps before the measurements start
N_flips = 10        # Number of steps between measurements

E = []
M = []
for T in T_range:
    # Find equilibrium
    for _ in range(N_eq):
        iterate_system(s, T)
    # measure
    _E = []
    _M = []
    for _ in range(N_sweeps):
        _E.append(s.E)
        _M.append(s.M)
        for _ in range(N_flips):
            iterate_system(s, T)
    
    E.append(np.mean(_E))
    M.append(np.mean(_M))

plt.figure()
plt.subplot(1, 2, 1)
plt.plot(T_range, E)
plt.title("E")
plt.subplot(1, 2, 2)
plt.plot(T_range, M)
plt.title("M")
plt.tight_layout()
plt.show()